In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)




## Import libraries[](http://)

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
#from sklearn.externals.six import StringIO
from sklearn import tree
from six import StringIO
from sklearn.model_selection import KFold, cross_val_score
from sklearn import metrics


#visualization
%matplotlib inline
import seaborn as sns
from IPython.display import Image
#import pydotplus
from sklearn.tree import export_graphviz

## EDA

In [3]:
df = pd.read_csv('../input/arketing-campaign/marketing_campaign.csv', delimiter=';')

In [4]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0


In [5]:
df_cleaned = df.copy()

del df_cleaned['Z_CostContact']
del df_cleaned['Z_Revenue']


In [6]:
#create account age
df_cleaned['Dt_Customer'] = pd.to_datetime(df_cleaned['Dt_Customer'])
df_cleaned['Dt_Customer_year_month'] = df_cleaned['Dt_Customer'].dt.to_period("M")
df_cleaned['account_age'] = (pd.to_datetime("2014-12").year - df_cleaned['Dt_Customer_year_month'].dt.year)*12 + (pd.to_datetime("2014-12").month - df_cleaned['Dt_Customer_year_month'].dt.month)

In [7]:
del df_cleaned['Dt_Customer_year_month']
del df_cleaned['Dt_Customer']
df_cleaned.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,account_age
0,5524,1957,Graduation,Single,58138.0,0,0,58,635,88,...,4,7,0,0,0,0,0,0,1,27
1,2174,1954,Graduation,Single,46344.0,1,1,38,11,1,...,2,5,0,0,0,0,0,0,0,9
2,4141,1965,Graduation,Together,71613.0,0,0,26,426,49,...,10,4,0,0,0,0,0,0,0,16
3,6182,1984,Graduation,Together,26646.0,1,0,26,11,4,...,4,6,0,0,0,0,0,0,0,10
4,5324,1981,PhD,Married,58293.0,1,0,94,173,43,...,6,5,0,0,0,0,0,0,0,11


In [8]:
# create customer age
df_cleaned['Age'] = 2014 - df_cleaned["Year_Birth"]
del df_cleaned['Year_Birth']
df_cleaned.head()

,ID,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,account_age,Age
0,5524,Graduation,Single,58138.0,0,0,58,635,88,546,...,7,0,0,0,0,0,0,1,27,57
1,2174,Graduation,Single,46344.0,1,1,38,11,1,6,...,5,0,0,0,0,0,0,0,9,60
2,4141,Graduation,Together,71613.0,0,0,26,426,49,127,...,4,0,0,0,0,0,0,0,16,49
3,6182,Graduation,Together,26646.0,1,0,26,11,4,20,...,6,0,0,0,0,0,0,0,10,30
4,5324,PhD,Married,58293.0,1,0,94,173,43,118,...,5,0,0,0,0,0,0,0,11,33


In [9]:
df_cleaned.describe()

,ID,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,account_age,Age
count,2240.000000,2216.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,...,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000
mean,5592.159821,52247.251354,0.444196,0.506250,49.109375,303.935714,26.302232,166.950000,37.525446,27.062946,...,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393,0.009375,0.149107,17.195089,45.194196
std,3246.662198,25173.076661,0.538398,0.544538,28.962453,336.597393,39.773434,225.715373,54.628979,41.280498,...,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976,0.096391,0.356274,6.639904,11.984069
min,0.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,18.000000
25%,2828.250000,35303.000000,0.000000,0.000000,24.000000,23.750000,1.000000,16.000000,3.000000,1.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,37.000000
50%,5458.500000,51381.500000,0.000000,0.000000,49.000000,173.500000,8.000000,67.000000,12.000000,8.000000,...,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000,44.000000
75%,8427.750000,68522.000000,1.000000,1.000000,74.000000,504.250000,33.000000,232.000000,50.000000,33.000000,...,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.000000,55.000000
max,11191.000000,666666.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,263.000000,...,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,29.000000,121.000000


In [10]:
# Fillna in Income by using avg value

avg_income = np.mean(df_cleaned.Income)

df_cleaned['Income'] = df_cleaned['Income'].fillna(avg_income, axis = 0)

In [11]:
df_cleaned.describe()

,ID,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,account_age,Age
count,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,...,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000
mean,5592.159821,52247.251354,0.444196,0.506250,49.109375,303.935714,26.302232,166.950000,37.525446,27.062946,...,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393,0.009375,0.149107,17.195089,45.194196
std,3246.662198,25037.797168,0.538398,0.544538,28.962453,336.597393,39.773434,225.715373,54.628979,41.280498,...,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976,0.096391,0.356274,6.639904,11.984069
min,0.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,18.000000
25%,2828.250000,35538.750000,0.000000,0.000000,24.000000,23.750000,1.000000,16.000000,3.000000,1.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,37.000000
50%,5458.500000,51741.500000,0.000000,0.000000,49.000000,173.500000,8.000000,67.000000,12.000000,8.000000,...,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000,44.000000
75%,8427.750000,68289.750000,1.000000,1.000000,74.000000,504.250000,33.000000,232.000000,50.000000,33.000000,...,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.000000,55.000000
max,11191.000000,666666.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,263.000000,...,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,29.000000,121.000000


In [12]:
df_cleaned.corr()

,ID,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,account_age,Age
ID,1.000000,0.013036,0.002406,-0.002580,-0.046524,-0.022878,0.004600,-0.004437,-0.024475,-0.007642,...,-0.007446,-0.036040,-0.025387,-0.007517,-0.021614,-0.015061,0.033883,-0.021968,-0.001554,-0.000028
Income,0.013036,1.000000,-0.425176,0.019018,-0.003946,0.576789,0.428747,0.577802,0.437497,0.436162,...,-0.549824,-0.016168,0.182791,0.334850,0.274921,0.087538,-0.027223,0.132756,-0.019852,0.160942
Kidhome,0.002406,-0.425176,1.000000,-0.036133,0.008827,-0.496297,-0.372581,-0.437129,-0.387644,-0.370673,...,0.447846,0.014674,-0.161600,-0.205634,-0.172339,-0.081716,0.040207,-0.080008,-0.053611,-0.230176
Teenhome,-0.002580,0.019018,-0.036133,1.000000,0.016198,0.004846,-0.176764,-0.261160,-0.204187,-0.162475,...,0.134884,-0.042677,0.038886,-0.191050,-0.140090,-0.015605,0.003138,-0.154446,0.017636,0.352111
Recency,-0.046524,-0.003946,0.008827,0.016198,1.000000,0.016064,-0.004306,0.023056,0.001079,0.022670,...,-0.021445,-0.032991,0.018826,0.000129,-0.019283,-0.001781,0.013231,-0.198437,0.024897,0.019871
MntWines,-0.022878,0.576789,-0.496297,0.004846,0.016064,1.000000,0.389637,0.562667,0.399753,0.386581,...,-0.320653,0.062202,0.373286,0.472613,0.354133,0.205907,-0.039007,0.247254,0.166528,0.157773
MntFruits,0.004600,0.428747,-0.372581,-0.176764,-0.004306,0.389637,1.000000,0.543105,0.594804,0.567164,...,-0.418383,0.014727,0.010152,0.215833,0.194748,-0.009773,-0.005166,0.125289,0.066266,0.017917
MntMeatProducts,-0.004437,0.577802,-0.437129,-0.261160,0.023056,0.562667,0.543105,1.000000,0.568402,0.523846,...,-0.539470,0.018272,0.102912,0.373769,0.309761,0.043033,-0.023483,0.236335,0.092375,0.030872
MntFishProducts,-0.024475,0.437497,-0.387644,-0.204187,0.001079,0.399753,0.594804,0.568402,1.000000,0.579870,...,-0.446003,0.000357,0.016843,0.199578,0.260762,0.002577,-0.020953,0.111331,0.080270,0.041625
MntSweetProducts,-0.007642,0.436162,-0.370673,-0.162475,0.022670,0.386581,0.567164,0.523846,0.579870,1.000000,...,-0.423294,0.001530,0.028641,0.259590,0.241818,0.009985,-0.022485,0.117372,0.081657,0.018133


In [13]:
#from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [14]:
#Create Dummy Variables
enc_edu = OneHotEncoder(handle_unknown='ignore')
enc_edu_df = pd.DataFrame(enc_edu.fit_transform(df_cleaned[['Education']]).toarray())

df_cleaned = df_cleaned.join(enc_edu_df)

df_cleaned.head()

,ID,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,AcceptedCmp2,Complain,Response,account_age,Age,0,1,2,3,4
0,5524,Graduation,Single,58138.0,0,0,58,635,88,546,...,0,0,1,27,57,0.0,0.0,1.0,0.0,0.0
1,2174,Graduation,Single,46344.0,1,1,38,11,1,6,...,0,0,0,9,60,0.0,0.0,1.0,0.0,0.0
2,4141,Graduation,Together,71613.0,0,0,26,426,49,127,...,0,0,0,16,49,0.0,0.0,1.0,0.0,0.0
3,6182,Graduation,Together,26646.0,1,0,26,11,4,20,...,0,0,0,10,30,0.0,0.0,1.0,0.0,0.0
4,5324,PhD,Married,58293.0,1,0,94,173,43,118,...,0,0,0,11,33,0.0,0.0,0.0,0.0,1.0


In [15]:
df_cleaned.rename(columns = {0: "2n Cycle", 1: "Basic", 2: "Graduation", 3: "Master", 4: "PhD"},inplace = True)

In [16]:
enc_marital = OneHotEncoder(handle_unknown='ignore')
enc_marital_df = pd.DataFrame(enc_edu.fit_transform(df_cleaned[['Marital_Status']]).toarray())

df_cleaned = df_cleaned.join(enc_marital_df)

df_cleaned.rename(columns = {0: "Absurd", 1: "Alone", 2: "Divorced", 3: "Married", 4: "Single", 5: "Together", 6: "Widow", 7: "YOLO"}, inplace = True)
df_cleaned.head()

,ID,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,Master,PhD,Absurd,Alone,Divorced,Married,Single,Together,Widow,YOLO
0,5524,Graduation,Single,58138.0,0,0,58,635,88,546,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2174,Graduation,Single,46344.0,1,1,38,11,1,6,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,4141,Graduation,Together,71613.0,0,0,26,426,49,127,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,6182,Graduation,Together,26646.0,1,0,26,11,4,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5324,PhD,Married,58293.0,1,0,94,173,43,118,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [17]:
del df_cleaned['Marital_Status']
del df_cleaned['Education']

In [18]:
df_cleaned["total_Mnt"] = df_cleaned["MntWines"] + df_cleaned["MntFruits"] + df_cleaned["MntMeatProducts"]+ df_cleaned['MntFishProducts'] + df_cleaned["MntSweetProducts"] + df_cleaned["MntGoldProds"]

df_cleaned['MntWines_pct'] = df_cleaned['MntWines']/df_cleaned['total_Mnt']
df_cleaned['MntFruits_pct'] = df_cleaned["MntFruits"]/df_cleaned['total_Mnt']
df_cleaned["MntMeatProducts_pct"] = df_cleaned["MntMeatProducts"]/df_cleaned['total_Mnt']
df_cleaned["MntFishProducts_pct"] = df_cleaned["MntFishProducts"]/df_cleaned['total_Mnt']
df_cleaned["MntSweetProducts_pct"] = df_cleaned["MntSweetProducts"]/df_cleaned['total_Mnt']
df_cleaned["MntGoldProds_pct"] = df_cleaned["MntGoldProds"]/df_cleaned['total_Mnt']


In [19]:
df_cleaned[['MntWines_pct', 'MntFruits_pct', 'MntMeatProducts_pct', 'MntFishProducts_pct', 'MntSweetProducts_pct', 'MntGoldProds_pct']].describe()

,MntWines_pct,MntFruits_pct,MntMeatProducts_pct,MntFishProducts_pct,MntSweetProducts_pct,MntGoldProds_pct
count,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000
mean,0.458481,0.049505,0.249508,0.071532,0.050777,0.120196
std,0.228411,0.055867,0.126633,0.077909,0.061184,0.108988
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.289506,0.008924,0.156250,0.012571,0.008621,0.038095
50%,0.457482,0.029840,0.233447,0.048193,0.033333,0.086294
75%,0.639143,0.070237,0.328227,0.104703,0.070411,0.169768
max,0.963303,0.445545,0.997110,0.590909,0.945848,0.894150


In [20]:
# Create AcceptedCmps to count number of offers each customer accepted
df_cleaned["AcceptedCmps"] = df_cleaned[["AcceptedCmp1","AcceptedCmp2","AcceptedCmp3","AcceptedCmp4", "AcceptedCmp5"]].sum(axis = 1)

In [21]:
df_cleaned_for_DS = df_cleaned.copy()

## Start to Create Random Forest

In [22]:
#Balanced dataset by oversampling
count_response_0, count_response_1 = df_cleaned_for_DS.Response.value_counts()
print(count_response_0)
print(count_response_1)

df_cleaned_for_DS_0 = df_cleaned_for_DS[df_cleaned_for_DS["Response"] == 0]
df_cleaned_for_DS_1 = df_cleaned_for_DS[df_cleaned_for_DS["Response"] == 1]

df_cleaned_for_DS_1_over = df_cleaned_for_DS_1.sample(n = count_response_0, replace = True)
df_cleaned_for_DS_1_over.shape

1906
334


(1906, 46)

In [23]:
df_cleaned_for_DS_resampling = pd.concat([df_cleaned_for_DS_1_over, df_cleaned_for_DS_0], axis = 0)

In [24]:
# split the train and test dataset

X = df_cleaned_for_DS_resampling[feature]
y = df_cleaned_for_DS_resampling.Response

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.4, random_state = 123)

NameError: name 'feature' is not defined

In [25]:
feature = ["2n Cycle", "Basic", "Graduation", "Master", "PhD", "Absurd", "Alone", "Divorced", "Married", "Single",
            "Together", "Widow", "YOLO", "Income", "Kidhome", "Teenhome", "Recency", "MntWines_pct", "MntFruits_pct",
            "MntMeatProducts_pct", "MntFishProducts_pct", "MntSweetProducts_pct", "MntGoldProds_pct", "NumDealsPurchases",
            "NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases", "NumWebVisitsMonth",
            "AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5",
            "AcceptedCmps", "Complain",
            "account_age", "Age"] 

In [26]:
from sklearn.model_selection import GridSearchCV

## Use GridSearchCV to find the optimized tree depth

In [27]:
rfc = RandomForestClassifier(n_estimators=30,
                             random_state=1)

max_depth_range = range(1,16)
param_grid = dict(max_depth=max_depth_range)

grid = GridSearchCV(rfc,
                    param_grid,
                    cv = 10,
                    scoring = 'accuracy')

grid.fit(X_train, y_train)


grid.cv_results_

NameError: name 'X_train' is not defined

In [28]:
grid_mean_scores = grid.cv_results_["mean_test_score"]
grid_test_mean_scores = metrics
# plot the results
sns.mpl.pyplot.plot(max_depth_range, grid_mean_scores)
sns.mpl.pyplot.xlabel('max_depth')
sns.mpl.pyplot.ylabel('Cross-Validated Mean Train Set Accuracy');

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [29]:
best_rfc = RandomForestClassifier(n_estimators=50,
                                  random_state=1,
                                  max_depth = 12)
best_rfc.fit(X_train, y_train)

rfc_pred = best_rfc.predict(X_test)
accuracy_train = metrics.accuracy_score(y_train, best_rfc.predict(X_train))
accuracy_test = metrics.accuracy_score(y_test, rfc_pred)
print("Accuracy of Random Forest train is: ", accuracy_train)
print("Accuracy of Random Forest test is: ", accuracy_test)

NameError: name 'X_train' is not defined

In [30]:
# compute feature importances
pd.DataFrame({'feature':feature, 'importance':best_rfc.feature_importances_}).sort_values(by='importance',ascending=False)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [31]:
DecisionTree_cross_val = DecisionTreeClassifier(max_depth=10, random_state=10)
DecisionTree_cross_val.fit(X_train, y_train)
cross_val_scores = cross_val_score(DecisionTree_cross_val, X_train, y_train, cv = 10)
print("10-fold accuracies:\n",cross_val_scores)
print("10-fold mean accuracy:\n",cross_val_scores.mean())

NameError: name 'X_train' is not defined

In [32]:
y_test_pred_rcf = best_rfc.predict(X_test)

NameError: name 'X_test' is not defined

In [33]:
print("Classification Report:\n", metrics.classification_report(y_test,y_test_pred_rcf))

NameError: name 'y_test' is not defined

In [34]:
pred_probs_rcf =  best_rfc.predict_proba(X_test)
pred_probs_rcf

NameError: name 'X_test' is not defined

In [35]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_probs_rcf[:,1])

NameError: name 'y_test' is not defined

In [36]:
sns.mpl.pyplot.plot(fpr, tpr,label="DS")
sns.mpl.pyplot.xlim([0, 1])
sns.mpl.pyplot.ylim([0, 1.05])
sns.mpl.pyplot.legend(loc="lower right")
sns.mpl.pyplot.xlabel('False Positive Rate (1 - Specificity)')
sns.mpl.pyplot.ylabel('True Positive Rate (Sensitivity)')

NameError: name 'fpr' is not defined

In [37]:
print("Test AUC: ",metrics.roc_auc_score(y_test, best_rfc.predict(X_test)))

NameError: name 'y_test' is not defined